# Visualizing Attention in Huggingface Transformers

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/labmlai/inspectus/blob/main/notebooks/gpt2.ipynb)

This Jupyter notebook demonstrates how to use the Inspectus library with Huggingface Transformers. In this case, we will be focusing on the GPT-2 model.

In [ ]:
!pip install inspectus
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoConfig, GPT2LMHeadModel
import inspectus
import torch

Following cell sets up a GPT-2 model from Huggingface's Transformers library. It initializes the tokenizer and model configuration, then creates the GPT-2 model.

In [ ]:
context_length = 128
tokenizer = AutoTokenizer.from_pretrained("huggingface-course/code-search-net-tokenizer")

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

This cell takes a sentence and tokenizes it using the previously initialized tokenizer. The tokenized output is then used to create input IDs for the model and a list of tokens. It uses the 'offset_mapping' attribute returned by the tokenizer to slice the original text into individual tokens.

In [ ]:
text= 'The quick brown fox jumps over the lazy dog'
tokenized = tokenizer(
    text,
    return_tensors='pt',
    return_offsets_mapping=True
)
input_ids = tokenized['input_ids']

tokens = [text[s: e] for s, e in tokenized['offset_mapping'][0]]

In [ ]:
with torch.no_grad():
    res = model(input_ids=input_ids.to(model.device), output_attentions=True)

The `attention` function from the Inspectus library is used to visualize the attention weights. The function takes the attention weights from the model output and the list of tokens as input. The `chart_types` parameter specifies the types of visualizations to be generated. The `color` parameter is used to specify the color palette for the visualizations.

In [ ]:
inspectus.attention(res['attentions'], tokens, 
          chart_types=['attention_matrix', 'query_token_heatmap', 'key_token_heatmap', 'dimension_heatmap', 'token_dim_heatmap', 'line_grid']
             ,color={
                 'query_token_heatmap': 'orange',
                 'key_token_heatmap': 'green',
                 'token_dim_heatmap': 'red',
             }
         )